-----------------------------------
#### **Programacion Entera con Python: Caso Warehouse location**
**Universidad Tecnologica Nacional - Facultad Buenos Aires** <br>
**Ingenieria Industrial**<br>
**Investigacion Operativa** <br>
Autor: Martin Palazzo + Caylie Cincera (https://www.youtube.com/watch?v=5I0mhX0973o) <br>
Curso I4051 <br>



In [20]:
pip install pulp

     |████████████████████████████████| 41.2 MB 78 kB/s 


In [37]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pulp
from pulp import LpProblem
from pulp import LpMinimize
from pulp import LpVariable
from pulp import LpBinary
from pulp import *

$$
\begin{matrix}
c_{ij} & = & \text{costo de transportar del centro j al cliente i} \\
x_{ij} & = &\text{el centro j provee al cliente i} \\
u_j & = &\text{costo fijo de construir el centro j} \\
w_j & = &\text{variable de existencia del centro j}
\end{matrix}
$$

$$
\begin{matrix}
\underset{x}{\text{min }} & \sum_{i = 1}^{n}\sum_{j = 1}^{m} c_{ij} x_{ij} + \sum_{j = 1}^{m} u_j w_j \\ 
& \\
\text{s.t. } & \sum_{j = 1}^{m} x_{ij} = 1 \\ 
\\
& x_{ij} \leq d_i w_j  \\
& x_{ij} \in \{ 0,1\} \\
& w_{j} \in \{ 0,1\}
\end{matrix}
$$

In [38]:
# definimos la cantidad de clientes
clientes = [1,2,3,4,5]
# definimos la cantidad de depositos
warehouses = ['w1','w2','w3']

# demanda de cada cliente
demanda = {1:80,
           2:270,
           3:250,
           4:160,
           5:180}

# costo de activar cada warehouse
wcost = {'w1':1000,'w2':1000,'w3':1000}

# maxima cantidad que puede mover cada warehouse
max_q = {'w1':500,'w2':500,'w3':500}

# costo de transporte warehouse-cliente
transp_c = {'w1': {1 : 4, 2:5 , 3:6 , 4:8, 5:10},
            'w2': {1 : 6, 2:4 , 3:3 , 4:5, 5:8},
            'w3': {1 : 9, 2:7 , 3:4 , 4:3, 5:4}}

In [45]:
# definir el problema en pulp
opt = LpProblem("warehouseLocation", LpMinimize)

# definir las variables de decision Xij
xij = LpVariable.dicts("Servicio", [(i,j) for i in clientes
                                                for j in warehouses], 0)
# definir la Wj
Uj = LpVariable.dicts("UsarLocacion", warehouses,0,1,LpBinary)

# funcion objetivo
opt += lpSum(wcost[j]*Uj[j] for j in warehouses) + lpSum(transp_c[j][i]*xij[(i,j)] for j in warehouses for i in clientes)

# restricciones 1
for i in clientes:
  opt += lpSum(xij[(i,j)] for j in warehouses) == demanda[i] 

# restruccion 1
for j in warehouses:
  opt += lpSum(xij[(i,j)] for i in clientes) <= max_q[j]*Uj[j] 

for i in clientes:
  for j in warehouses:
    opt += xij[(i,j)] <= demanda[i]*Uj[j]

In [46]:
# solucion
opt.solve()

1

In [47]:
# imprimir las variables de decision Wj 
tol = 0.00001
for i in warehouses:
  if Uj[i].varValue > tol:
    print("Construir un warehouse en el sitio ",i)

Construir un warehouse en el sitio  w2
Construir un warehouse en el sitio  w3


In [48]:
# imprimir las variables de decision Xij
for q in opt.variables():
  print(q.name,"=",q.varValue)

Servicio_(1,_'w1') = 0.0
Servicio_(1,_'w2') = 80.0
Servicio_(1,_'w3') = 0.0
Servicio_(2,_'w1') = 0.0
Servicio_(2,_'w2') = 270.0
Servicio_(2,_'w3') = 0.0
Servicio_(3,_'w1') = 0.0
Servicio_(3,_'w2') = 150.0
Servicio_(3,_'w3') = 100.0
Servicio_(4,_'w1') = 0.0
Servicio_(4,_'w2') = 0.0
Servicio_(4,_'w3') = 160.0
Servicio_(5,_'w1') = 0.0
Servicio_(5,_'w2') = 0.0
Servicio_(5,_'w3') = 180.0
UsarLocacion_w1 = 0.0
UsarLocacion_w2 = 1.0
UsarLocacion_w3 = 1.0


In [49]:
# imprimir la funcion objetivo en el optimo
print('El costo de la operacion es de = ', value(opt.objective))

El costo de la operacion es de =  5610.0
